In [1]:
#Se instalan todas las dependencias necesarias para el proyecto (vedo, trimesh, open3d, pandas)
!pip install vedo trimesh open3d pandas

In [2]:
import vedo
import trimesh
import open3d as o3d
import pandas as pd
import numpy as np
import os
import imageio

#Función load_points: Permite cargar los puntos a partir de un archivo verificando que sea csv o json
#También pone una lista de puntos por defecto en caso de que no se específique la ruta del archivo a cargar
def load_points(file_path=None):
    if file_path:
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        elif file_path.endswith('.json'):
            df = pd.read_json(file_path)
        else:
            raise ValueError("Archivo no soportado")
        return df[['x', 'y', 'z']].values
    else:
        return np.array([[0,0,0], [1,2,1], [2,0,3], [3,3,0], [4,2,2]])

#Se llama la función load_points, aquí podemos pasar la ruta del archivo a cargar
points = load_points()  

#Función create__objects_from_points: Con ayuda de la librería vedo toma los puntos y crea objetos
''' 1. Los objetos en el índice 0 y en índices 3i con i>=0 serán cubos.
    2. Los objetos en la posición 1 y en índices 3i+1 con i>=0 serán esferas.
    3. Los objetos en la posición 2, y en índices 3i+2 con i>=1 serán cilindros.
'''
def create_objects_from_points(points):
    objects = []
    for i, (x, y, z) in enumerate(points):
        if i % 3 == 0:
            obj = vedo.Cube(pos=(x, y, z), side=0.5 + i * 0.1, c='red')
        elif i % 3 == 1:
            obj = vedo.Sphere(pos=(x, y, z), r=0.3 + 0.05*i, c='blue')
        else:
            obj = vedo.Cylinder(pos=(x, y, z), r=0.2 + 0.05*i, height=0.6 + 0.2*i, c='green')
        objects.append(obj)
    return objects
#Los objetos generados según las funciones de vedo y los puntos se retornan en una lista

#La lista retornada al llamar la función anterior, se guardan en la variable shapes
shapes = create_objects_from_points(points)

#Con el método show de vedo se pueden poner las figuras en la escena de acuerdo a ciertos parámetros
vedo.show(shapes, axes=1, viewup="z", bg='white')

#En este bloque se van a realizar transformaciones, y se van a exportar frames para formar un gif
'''Las transformaciones que se hacen son principalmente la aplicación de escala y el cambio de color,
además se usa una variable t, que está sujeta a la cantidad de frames para que se aplique una
transformación por cada frame realizado'''
n_frames = 20
plotter = vedo.Plotter(offscreen=True, bg='white')
frame_dir = "frames"
os.makedirs(frame_dir, exist_ok=True)
filenames = []

for t in range(n_frames):
    for i, obj in enumerate(shapes):
        scale = 1 + 0.2 * np.sin(t * 0.3 + i)
        obj.scale([scale]*3)
        obj.color(['red', 'blue', 'green'][i % 3])
    
    plotter.clear()
    plotter.show(shapes, resetcam=(t == 0), interactive=False)
    
    frame_path = os.path.join(frame_dir, f"frame_{t:03d}.png")
    plotter.screenshot(frame_path)
    filenames.append(frame_path)

#Se toman los frames realizados anteriormente para crear y exportar el gif
gif_path = "animacion.gif"
with imageio.get_writer(gif_path, mode='I', duration=0.1, loop=0) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

print(f"GIF exportado como {gif_path}")

#Finalmente, utilizando el método write de vedo, podemos por cada figura exportar un archivo stl 
def export_objects_vedo(shapes, folder="exported", fmt="obj"):
    os.makedirs(folder, exist_ok=True)
    for i, shape in enumerate(shapes):
        path = os.path.join(folder, f"shape_{i}.{fmt}")
        vedo.write(shape, path)
    print(f"Exportado con vedo a .{fmt}")

export_objects_vedo(shapes, fmt="stl")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


C:\Users\judar\AppData\Local\Temp\ipykernel_30504\3575893412.py:77: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


GIF exportado como animacion.gif
Exportado con vedo a .stl
